<a href="https://colab.research.google.com/github/hananajiyya/Python-Practice/blob/Others%2C-Deep-Learning%2C-Machine-Learning%2C-Models/T5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install transformers
!pip install transformers datasets
!pip install rouge-score nltk
!pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!apt install git-lfs
!git config --global user.email "hana.najiyya@gmail.com"
!git config --global user.name "hananajiyya"

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.


In [ ]:
import transformers

print(transformers.__version__)

4.19.2


In [ ]:
model_checkpoint = "t5-large"

In [ ]:
from datasets import load_dataset, load_metric

raw_datasets = load_dataset("csebuetnlp/xlsum",'indonesian')
metric = load_metric("rouge")

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text'],
        num_rows: 38242
    })
    test: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text'],
        num_rows: 4780
    })
    validation: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text'],
        num_rows: 4780
    })
})

In [ ]:
raw_datasets["test"][0]

{'id': '141105_seni_vangogh',
 'summary': 'Sebuah mahakarya Vincent van Gogh, yang dilukis pada akhir kehidupannya, terjual di New York sebesar US$61,8 juta atau Rp752 miliar.',
 'text': 'Still Life, Vase with Daisies, and Poppies dibuat beberapa bulan sebelum van Gogh meninggal. Still Life, Vase with Daisies, and Poppies melampaui perkiraan sebelumnya US$50 juta pada pelelangan di Sotheby. Lukisan Van Gogh dibuat di rumah dokter Prancisnya hanya beberapa bulan sebelum dirinya meninggal di tahun 1890. Karyanya ini adalah satu dari beberapa lukisannya yang terjual saat Van Gogh masih hidup. Rekor penjualan pelukis Belanda pada balai lelang adalah US$82,5 juta lewat Portrait of Dr Gachet. Lukisan tentang dokternya ini terjual pada tahun 1990. Pada pelelangan ini patung Alberto Giacometti dari tahun1951 terjual seharga lebih dari US$100 juta, di bawah rekor karya seniman Swiss itu senilai US$104,3 juta. Sementara pematung Amedeo Modigliani mencatat rekor baru lewat Tete patung kepala yang

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML


def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(
        dataset
    ), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    df = df[['summary','text']]
    df.dropna()
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(raw_datasets["test"])

In [ ]:
metric

Metric(name: "rouge", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}, usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLSum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/datasets/issues/617
    use_stemmer: Bool indicating whether Porter stemmer should be used to strip word suffixes.
    use_aggregator: Return aggregates if this is set to True
Retu

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:161: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [ ]:
if model_checkpoint in ["t5-small", "t5-base", "t5-large", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

In [ ]:
tokenizer.model_max_length

512

In [ ]:
max_input_length = 1024
max_target_length = 512


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=tokenizer.model_max_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["summary"], max_length=tokenizer.model_max_length, truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
preprocess_function(raw_datasets["train"][:2])

{'input_ids': [[21603, 10, 2043, 8478, 127, 3304, 146, 9, 42, 1468, 3, 449, 40, 1598, 9, 123, 157, 413, 142, 18956, 6, 3, 7, 1194, 2551, 108, 7, 6820, 9, 8411, 9, 107, 10823, 49, 4243, 15, 107, 3304, 4353, 4612, 142, 1625, 9, 107, 1076, 26, 9, 4665, 3304, 399, 7396, 17, 152, 5, 3953, 7922, 1191, 107, 3, 63, 1468, 1076, 8758, 1598, 34, 76, 10447, 17, 152, 16686, 1227, 783, 78, 7, 23, 138, 5, 925, 7922, 1191, 107, 3, 1152, 40, 1665, 3, 17, 1639, 5748, 76, 3, 2551, 107, 6, 1076, 9, 1939, 157, 7637, 17, 9, 1076, 122, 77, 1191, 157, 142, 115, 9, 22898, 158, 7, 49, 17, 9, 95, 9, 1720, 9, 5, 6619, 1639, 3, 354, 12204, 10, 5622, 440, 1227, 8044, 9, 3464, 4550, 63, 15, 12534, 7922, 1191, 107, 3, 4849, 15, 2780, 3, 17, 23, 115, 9, 18, 17, 23, 115, 9, 1076, 8758, 1598, 5, 3963, 12204, 7922, 1191, 107, 34, 76, 3, 1258, 2782, 3, 32, 109, 107, 1394, 76, 144, 76, 1227, 46, 2046, 9, 3856, 158, 7, 49, 17, 9, 3, 3768, 4550, 8765, 22498, 5, 3159, 142, 15076, 144, 140, 8478, 127, 3304, 7922, 1191, 107, 50

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

# **Fine-Tuning the model**

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq

model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
batch_size = 8
learning_rate = 2e-5
weight_decay = 0.01
num_train_epochs = 5

# model_name = model_checkpoint.split("/")[-1]
# push_to_hub_model_id = f"assami-{model_name}-finetuned-xsum"

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [ ]:
tokenized_datasets["validation"]

In [ ]:
train_dataset = tokenized_datasets["test"].to_tf_dataset(
    batch_size=batch_size,
    columns=["input_ids", "attention_mask", "labels"],
    shuffle=True,
    collate_fn=data_collator,
)
validation_dataset = tokenized_datasets["validation"].to_tf_dataset(
    batch_size=8,
    columns=["input_ids", "attention_mask", "labels"],
    shuffle=False,
    collate_fn=data_collator,
)
generation_dataset = (
    tokenized_datasets["validation"]
    .shuffle()
    .select(list(range(200)))
    .to_tf_dataset(
        batch_size=8,
        columns=["input_ids", "attention_mask", "labels"],
        shuffle=False,
        collate_fn=data_collator,
    )
)

In [ ]:
from transformers import AdamWeightDecay
import tensorflow as tf

optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer)

In [ ]:
import numpy as np
import nltk


def metric_fn(eval_predictions):
    predictions, labels = eval_predictions
    decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    for label in labels:
        label[label < 0] = tokenizer.pad_token_id  # Replace masked label tokens
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Rouge expects a newline after each sentence
    decoded_predictions = [
        "\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_predictions
    ]
    decoded_labels = [
        "\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels
    ]
    result = metric.compute(
        predictions=decoded_predictions, references=decoded_labels, use_stemmer=True
    )
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    # Add mean generated length
    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions
    ]
    result["gen_len"] = np.mean(prediction_lens)

    return result

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
# from transformers.keras_callbacks import PushToHubCallback, KerasMetricCallback
# from tensorflow.keras.callbacks import TensorBoard
from transformers.keras_callbacks import KerasMetricCallback

# tensorboard_callback = TensorBoard(log_dir="./summarization_model_save/logs")

# push_to_hub_callback = PushToHubCallback(
#     output_dir="./summarization_model_save",
#     tokenizer=tokenizer,
#     hub_model_id=push_to_hub_model_id,
# )

metric_callback = KerasMetricCallback(
    metric_fn, eval_dataset=generation_dataset, predict_with_generate=True
)

# callbacks = [metric_callback, tensorboard_callback, push_to_hub_callback]

model.fit(
    train_dataset, validation_data=validation_dataset, epochs=3, verbose=2, callbacks=[metric_callback]
)